In [7]:
import pandas as pd
import numpy as np
import pickle
import time 
import os
from datetime import datetime
from operator import itemgetter

data_folder_path = '../'
raw_data_path = "../raw_data"
storage_path = "../database_storage"
daily_path = "../daily"
DTBS_path = os.path.join(storage_path, "DTBS.pkl")
DCBS_path = os.path.join(storage_path, "DCBS.pkl")
# re-open

with open(DCBS_path, 'rb') as f:  
    DCBS = pickle.load(f)
# re-open
with open(DTBS_path, 'rb') as f:  
    DTBS = pickle.load(f)

# 本文件的主要作用是读取DCBS.pkl，并将其转化成适合处理的DataFrame形式存储起来，需要在每天两点自动更新之后运行本文件

In [8]:
DictIndexs = {'bond_code' : '可转债代码',
              'date' : '日期',
              'cpr' : '转股溢价率',
              'dp' : '收盘价（全价）',
              'dl' : '双低',
              'ytm' : '债券到期收益率',
              'bl' : '债券余额',
              'trt' : '换手率（日）',
              'yl' : '剩余期限（年）',
              'csp' : '转股价',
              'csv' : '转股价值',
              'cz' : '纯债价值',
              'pd' : '平底溢价率',
              'ia' : '当前是否活跃',
              'qs' : '当日是否满足强赎条件',
              'qs15' : '15天内满足强赎条件的累计天数',
              'qs30' : '30天内满足强赎条件的累计天数',
              'qsn' : '满足强赎条件的累计天数',
              'xx' : '当日是否满足下修条件',
              'xx15' : '15天内满足下修条件的累计天数',
              'xx30' : '30天内满足下修条件的累计天数',
              'xxn' : '满足下修条件的累计天数',
              'rt1' : '发行人第一大股东持有比例',
              'ipo' : '上市日期',
              'sc' : '正股代码',
              'sn' : '正股中文名',
              'cat1' : '申万一级分类',
              'cat2' : '申万二级分类',
              'cl' : '正股收盘价',
              'pb' : '正股PB',
              'cn' : '转债中文名'
             }

# 对于三级字典，可以将前两级组成MultiIndex,然后使用pandas的from_dict创建Dataframe
DCBS_newDict = {(j, i): DCBS[i][j] for i in DCBS.keys() for j in DCBS[i].keys()}

DCBS_DataFrame = pd.DataFrame.from_dict(DCBS_newDict,orient='index')


# 重设DataFrame的索引，并将新增的两列改名，再重设索引
DCBS_DataFrame = DCBS_DataFrame.reset_index().copy()
DCBS_DataFrame.rename(columns={'level_0' : 'bond_code','level_1' : 'date'},inplace=True)

DCBS_DataFrame['id'] = DCBS_DataFrame['bond_code'] + ',' + DCBS_DataFrame['date']

DCBS_DataFrame.set_index('id', inplace=True)

print(DCBS_DataFrame)

                      bond_code        date     cpr       dp       dl    ytm  \
id                                                                             
123118.SZ,2023-04-27  123118.SZ  2023-04-27   -0.27  228.800  228.530 -14.33   
110057.SH,2023-04-27  110057.SH  2023-04-27   -0.22  144.579  144.359 -14.22   
127021.SZ,2023-04-27  127021.SZ  2023-04-27    0.00  112.549  112.549    NaN   
110053.SH,2023-04-27  110053.SH  2023-04-27    0.25  126.954  127.204  -5.43   
113044.SH,2023-04-27  113044.SH  2023-04-27    0.43  116.015  116.445  -0.64   
...                         ...         ...     ...      ...      ...    ...   
118010.SH,2023-08-25  118010.SH  2023-08-25  229.96  104.300  334.260   2.04   
123013.SZ,2023-08-25  123013.SZ  2023-08-25  236.31  421.895  658.205 -80.31   
128041.SZ,2023-08-25  128041.SZ  2023-08-25  241.69  386.721  628.411 -80.11   
118005.SH,2023-08-25  118005.SH  2023-08-25  321.00  103.356  424.356   2.45   
113576.SH,2023-08-25  113576.SH  2023-08

In [9]:
# 保存到新的数据库中，分别存储成pkl格式和csv格式
f_save = open('database_ByZCG\\DCBS_DataFrame.pkl', 'wb')
pickle.dump(DCBS_DataFrame, f_save)
f_save.close()

DCBS_DataFrame.to_csv('database_ByZCG\\DCBS_DataFrame.csv',encoding="utf_8_sig")